Parts of Duffision model building:
1. Forward process
- Noise schedule (betas, alphas)
2. Denoising process
- UNET ( task specific blocks: time embeddings, residual connections, attention blocks)
- Additional image guidance
- LDM implementation (?)
3. Sampling
- Same UNET but using something on top to speed it up

#### Data and imports

In [6]:
! pip install diffusers
! pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/66/f6/a2f07a3f5385b37c45b8e14448b8610a8618dfad18ea437cb23b4edc50c5/torchvision-0.19.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached torchvision-0.19.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.0 kB)
  Obtaining dependency information for torch==2.4.1 from https://files.pythonhosted.org/packages/1f/34/c93873c37f93154d982172755f7e504fdbae6c760499303a3111ce6ce327/torch-2.4.1-cp311-none-macosx_11_0_arm64.whl.metadata
  Using cached torch-2.4.1-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached torchvision-0.19.1-cp311-cp311-macosx_11_0_arm64.whl (1.7 MB)
   ━━━━━━━━━━━━━━

In [7]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler, UNet2DModel
from matplotlib import pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

ImportError: cannot import name 'get_ctx' from 'torch.library' (/Users/antonkutsenko/anaconda3/lib/python3.11/site-packages/torch/library.py)

In [ ]:
dataset = torchvision.datasets.MNIST(root="mnist/", train=True, download=True, transform=torchvision.transforms.ToTensor())


NameError: name 'torchvision' is not defined

In [ ]:
x, y = next(iter(train_dataloader))
print('Input shape:', x.shape)
print('Labels:', y)
plt.imshow(torchvision.utils.make_grid(x)[0], cmap='Greys');

#### Forward process

In [ ]:
def corrupt(x, amount):
  """Corrupt the input `x` by mixing it with noise according to `amount`"""
  noise = torch.rand_like(x)
  amount = amount.view(-1, 1, 1, 1) # Sort shape so broadcasting works
  return x*(1-amount) + noise*amount 



In [4]:
import torch

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 5))
axs[0].set_title('Input data')
axs[0].imshow(torchvision.utils.make_grid(x)[0], cmap='Greys')

# Adding noise
amount = torch.linspace(0, 1, x.shape[0]) # Left to right -> more corruption
noised_x = corrupt(x, amount)

# Plotting the noised version
axs[1].set_title('Corrupted data (-- amount increases -->)')
axs[1].imshow(torchvision.utils.make_grid(noised_x)[0], cmap='Greys');

Sources of inspiration:

- https://github.com/huggingface/diffusion-models-class
- https://nn.labml.ai/diffusion

